## Tutorial
### Credentials
[How to generate certificates](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Non-Interactive+%28bot%29+login#Non-Interactive(bot)login-CreatingaSelfSignedCertificate)

In [1]:
import os
from betfairlightweight.apiclient import APIClient

# If you do not have you credentials in your environment replace
# the strings after or with your own credentials.

USERNAME=os.environ.get("BETFAIR_USERNAME") or "BETFAIR_USERNAME"
PASSWORD=os.environ.get("BETFAIR_PASSWORD")  or "BETFAIR_PASSWORD"
APP_KEY=os.environ.get("BETFAIR_APP_KEY") or "BETFAIR_APP_KEY"
CERT_PATH=os.path.abspath("../certs") or "YOUR_ABSOLUTE_CERT_PATH"

LOCALE=None # Add locale here if you're are in any of
            # these jurisdictions [australia, italy, spain, romania, sweden]

trading: APIClient = APIClient(username=USERNAME, password=PASSWORD, app_key=APP_KEY, certs=CERT_PATH, locale=LOCALE)
trading.login()

<LoginResource>

###  Imports

In [2]:
import logging
import time

import numpy as np
import orjson

from betfairstreamer.models.betfair_api import BetfairMarketFilter, BetfairMarketDataFilter
from betfairstreamer.models.market_cache import MarketCache
from betfairstreamer.models.order_cache import OrderCache
from betfairstreamer.stream.betfair_connection_pool import BetfairConnectionPool
from betfairstreamer.utils import create_market_subscription, create_order_subscription

### Logging properties

In [3]:
np.set_printoptions(precision=3)
logging.basicConfig(level=logging.INFO)

### Create subscriptions.

This will specify what type of markets you want to subscribe to. Read more at
[MarketSubscription](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-MarketSubscriptionMessage),
[OrderSubscription](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-OrderSubscriptionMessage).

In [4]:
horse_subscription = create_market_subscription(
        # Asian Handicap is not yet supported if using EX_ALL_OFFER.
        market_filter=BetfairMarketFilter(eventTypeIds=["7"], marketTypes=["WIN"]),
        market_data_filter=BetfairMarketDataFilter(
            ladderLevels=3,  # Ladder levels are fixed to 3 at the moment.
            fields=[
                "EX_MARKET_DEF",
                "EX_BEST_OFFERS",
                "EX_LTP",
                "EX_BEST_OFFERS_DISP",
                "EX_ALL_OFFERS",
            ],
        ),
        conflate_ms=5000
)

order_subscription_message = create_order_subscription()

### Cache

These will aggregate the messages coming from the streams you have subscribed at. We need to do this because betfair will only send
the parts that have recently updated (deltas). So we need a way to keep the whole history.

In [5]:
market_cache = MarketCache()
order_cache = OrderCache()

### Connection Pool
For each of your subscription you have created __create_connection_pool__ will create a separate stream to betfair.
connection_pool will receive messages from each stream as they come in.

In [10]:
connection_pool = BetfairConnectionPool.create_connection_pool(subscription_messages=[horse_subscription, order_subscription_message],
                                                               app_key=trading.app_key,
                                                               session_token=trading.session_token)

INFO:root:b'{"op":"connection","connectionId":"205-280420030016-1166215"}'
INFO:root:b'{"op":"status","id":1,"statusCode":"SUCCESS","connectionClosed":false,"connectionsAvailable":9}'
INFO:root:b'{"op":"status","id":1,"statusCode":"SUCCESS","connectionClosed":false}'
INFO:root:b'{"op":"connection","connectionId":"206-280420030017-1171408"}'
INFO:root:b'{"op":"status","id":1,"statusCode":"SUCCESS","connectionClosed":false,"connectionsAvailable":8}'
INFO:root:b'{"op":"status","id":1,"statusCode":"SUCCESS","connectionClosed":false}'


### Read loop

1: __connection_pool.read()__ is a generator which will produce messages as long as betfair does not disconnect or your internet connection goes down.
Since the messages we receive are bytes we need to decode them,

2: __orjson.loads([bytes])__ produces python __dict__, which can be consumed by our caches.

3: __market_update__, __order_updates__, if  there were any market or order updates in the update, these will be added to respective list.

4: Iterate over both list to check if there are any updates, send them to your strategies and trade :)

### Market Book

![img](selections.png)

In the image we have a market with __three__ selections. Shakhter, Dinamo and Draw. That is each __row__. We have __three__ ladder levels
for Back/Lay each. That is the __columns__. We have __two__ different sides Back/Lay and For each ladder level (single column) we have a  price and size (__two__).

If you do not know about numpy you will likely want to read up on that before using this library since everything builds on that.

Now market book represent all of these as a numpy ndarray and specifically in this case a ndarray with axes (3, 2, 3, 2)

The first axis is the sort priority (0/1/2), second axis which side to select BACK/LAY (0/1), third axis ladder level (0, 1, 2) and
last axis price and size (0/1)

#### Example

If i want the best __back__ price for __Shakhter__
```python
market_book.best_display[0, 0, 0, 0]
```
If i want the best __lay__ price for __The draw__
```python
market_book.best_display[2, 1, 0, 0]
```

If i want second best back prices for all selections
```python
market_book.best_display[:, 0, 1, 0]
```

### Other notes.

market_book.best_display have the same prices as betfair homepage, read more about why this might not be the case at
[Cross matching](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Additional+Information#AdditionalInformation-VirtualBets).

Learn how to slice numpy arrays. You can do a lot of cool tricks and fast vectorized operations on the markets array.

In [ ]:
try:
    for update in connection_pool.read():
        update = orjson.loads(update)

        market_updates = market_cache(update)
        order_updates = order_cache(update)

        for market_book in market_updates:
            print(f"{market_book.market_id} {round(time.time() - market_cache.publish_time/1000, 4)} {market_book.best_display[0, 0, :, 0]}\n", end='\r')

        for order in order_updates:
            print(order)
except Exception as _:
    pass
finally:
    for k, v in connection_pool.connections.items():
        v.connection.close()

1.170343078 5.0868 [6.  5.9 5.8]
1.170343245 5.0871 [4.4 4.3 4.2]
1.170343082 5.0873 [36. 30. 29.]
1.170343088 5.0874 [6.4 6.  5.9]
1.170343084 5.0876 [7.6 7.4 7.2]
1.170343257 5.0877 [13.5  9.8  6.8]
1.170343086 5.0884 [9.8 9.4 9.2]
1.170343080 5.0885 [7.4 7.2 6.8]
1.170343092 5.0889 [7.4 7.2 6. ]
1.170343090 5.089 [16.  15.5 14. ]
1.170343581 5.0892 [1.18 1.15 1.07]
1.170343583 5.0893 [1.3  1.15 1.08]
1.170343591 5.0895 [1.4  1.39 1.37]
1.170343251 5.0896 [3.6  3.55 3.5 ]
1.170343249 5.0897 [2.9  2.88 2.86]
1.170343247 5.0899 [8.4 8.  7.8]
1.170343589 5.09 [1.15 1.04 1.03]
1.170343255 5.0907 [8.  7.  5.7]
1.170343341 5.0909 [2.   1.25 1.1 ]
1.170343587 5.091 [1.6  1.15 1.1 ]
1.170343585 5.0913 [1.6  1.17 1.1 ]
1.170343333 5.0915 [2.   1.25 1.1 ]
1.170349305 5.0917 [34. 24.  6.]
1.170349302 5.0918 [8.8 7.2 5.8]
1.170349299 5.0919 [24. 18. 14.]
1.170349296 5.0921 [8.8 7.2 5.8]
1.170349293 5.0922 [10.   8.4  6.8]
1.170349287 5.0924 [6.  5.2 4.4]
1.170349284 5.0926 [14.5 10.5  8.6]
1.170

## Read historical data

In [ ]:
import bz2 # If you download data from betfair

market_cache = MarketCache()

market_updates = bz2.open("market_file.bz2", "r").readlines()  # If files are compressed

for market_update in market_updates:
    market_books = market_cache(orjson.loads(market_update))

    for market_book in market_books:
        pass